In [ ]:
from libraries import *
from parameters import *

In [ ]:
os.getcwd()
os.chdir(projectDir)

In [ ]:
adata = sc.read(par_save_filename_5)
adata.X.shape

In [ ]:
adata.uns['AllExpGeneNames'] = pd.Series(adata.var_names)

Select the genes that will be tested for their change upon the perturbation based on their expression profiles in the whole cell population. 

In [ ]:
sc.pp.filter_genes(adata, min_cells=par_mincells_for_testedgenes)
sc.pp.filter_cells(adata, min_genes=par_mincellgenes_for_testedgenes)

In [ ]:
fBarMat = adata.obs[adata.uns['feature_barcode_names']]

Discard guides that are available in less than "par_ncell_test_threshold" cells.

In [ ]:
badFBarMat = fBarMat.loc[:,fBarMat.sum(axis=0) < par_ncell_test_threshold]
badGuides = badFBarMat.columns.to_list()
badGDF = pd.DataFrame(data={'guides':badGuides})
badGDF['targetGene'] = badGDF['guides'].str.split("_", n=1, expand=True)[0]

badControlGuides = pd.read_csv(par_outlier_controlguides_file)
badControlGuides['targetGene'] = badControlGuides['OutlierGuides'].str.split("_", n=1, expand=True)[0]
badControlGuides = badControlGuides.rename(columns={"OutlierGuides": "guides"})
badGDF = pd.concat([badGDF, badControlGuides])
badGDF = badGDF.sort_values('guides')

In [ ]:
clusterSelectedBarcodes = adata.uns['feature_barcode_names']
clusterSelectedBarcodes = clusterSelectedBarcodes[~pd.Series(clusterSelectedBarcodes).isin(badGDF.guides.to_list())]
adata.uns['feature_barcode_names_filtered'] = clusterSelectedBarcodes

In [ ]:
fBarMat = adata.obs[adata.uns['feature_barcode_names_filtered']]
fBarMat[fBarMat>0] = 1
adata = adata[fBarMat.sum(axis=1) > 0,:]
adata.uns['feature_KO_barcode_names_filtered'] = [ x for x in adata.uns['feature_barcode_names_filtered'] if not (x.startswith("ONE_NONGENE_SITE") or x.startswith("NO_TARGET"))]


In [ ]:
adata.write(par_save_filename_7)